<a href="https://colab.research.google.com/github/bjih1999/cifar_img_classify_project/blob/master/VGGModelVer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import VGG16
from keras_preprocessing import image

IMAGE_SIZE = 32

CHANNEL_SIZE = 3

cifar10 = datasets.cifar10
# 데이터셋 다운로드 및 준비
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.reshape((50000, 32, 32, 3))
test_images = test_images.reshape((10000, 32, 32, 3))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

# 합성곱 층 만들기
# 아래 6줄의 코드에서 Conv2D와 MaxPooling2D 층을 쌓는 일반적인 패턴으로 합성곱 층을 정의합니다.
# CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다.
# MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만 컬러 이미지는 (R,G,B) 세 개의 채널을 가집니다
# 이 예에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의하겠습니다. 이 값을 첫 번째 층의 input_shape 매개변수로 전달합니다.

# 참고 : https://dryjelly.tistory.com/147
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNEL_SIZE))


model = models.Sequential()

for layer in vgg_model.layers[:-1]:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False

model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())

# Dense 층 추가하기
# 모델을 완성하려면 마지막 합성곱 층의 출력 텐서(크기 (4, 4, 64))를 하나 이상의 Dense 층에 주입하여 분류를 수행합니다.
# Dense 층은 벡터(1D)를 입력으로 받는데 현재 출력은 3D 텐서입니다.
# 먼저 3D 출력을 1D로 펼치겠습니다.
model.add(layers.Flatten())
# 그다음 하나 이상의 Dense 층을 그 위에 추가하겠습니다.
model.add(layers.Dense(64, activation='relu'))
# MNIST 데이터는 10개의 클래스가 있으므로 마지막에 Dense 층에 10개의 출력과 소프트맥스 활성화 함수를 사용합니다.
model.add(layers.Dense(10, activation='softmax'))
'''
ayer_names = [layer.name for layer in model.layers]
layer_outputs = [layer.output for layer in model.layers]

feature_map_model = tf.keras.models.Model(inputs=model.inputs, outputs=layer_outputs)
image_path= r"cat.jpg"
img = image.load_img(image_path, target_size=(150, 150))
input = image.img_to_array(img)
input = input.reshape((1,) + input.shape)
input = x.reshape((1,) + x.shape)
input /= 255.0
feature_maps = feature_map_model.predict(input)
for layer_name, feature_map in zip(layer_names, feature_maps):
    print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")
'''
# 모델 컴파일 및 훈련
# 옵티마이저, 손실함수, 지표

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 학습시킬 문제와 정답, 훈련 횟수
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.25)

# 모델 평가
# 풀게 할 문제와 정답
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_loss), print(test_acc)

# model.summary() 모델 구조 출력


58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/10
1172/1172 [==============================] - 47s 14ms/step - loss: 1.3657 - accuracy: 0.5204 - val_loss: 1.2127 - val_accuracy: 0.5729
Epoch 2/10
1172/1172 [==============================] - 16s 13ms/step - loss: 1.1656 - accuracy: 0.5901 - val_loss: 1.1867 - val_accuracy: 0.5825
Epoch 3/10
1172/1172 [==============================] - 16s 13ms/step - loss: 1.0732 - accuracy: 0.6212 - val_loss: 1.1549 - val_accuracy: 0.6056
Epoch 4/10
1172/1172 [==============================] - 16s 13ms/step - loss: 1.0009 - accuracy: 0.6475 - val_loss: 1.1388 - val_accuracy: 0.6030
Epoch 5/10
1172/1172 [==============================] - 16s 13ms/step - loss: 0.9326 - accuracy: 0.6706 - val_loss: 1.0890 - val_accuracy: 0.6254
Epoch 6/10
1172/1172 [==============================] - 16s 13ms/step - loss: 0.8717 - accuracy: 0.6918 - val_loss: 1.1154 - val_accuracy: 0.6173
Epoch 7/10
1172/1172 [==============================] - 16s